In [19]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv

load_dotenv()

True

In [20]:
llm = ChatGroq(
    temperature=0,
    model = "llama-3.3-70b-versatile"
)

response = llm.invoke("the first person to land on the moon was.....")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969, during the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to set foot on the moon.


In [21]:
# %pip install langchain langchain-community beautifulsoup4


In [22]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/job/R-33460")
page_data = loader.load().pop().page_content
print(page_data)






















Nike Careers









































Skip to main content
Open Virtual Assistant










Home


Career Areas


Total Rewards


Life@Nike


Purpose










Language





Select a Language

  Deutsch  
  English  
  Español (España)  
  Español (América Latina)  
  Français  
  Italiano  
  Nederlands  
  Polski  
  Tiếng Việt  
  Türkçe  
  简体中文  
  繁體中文  
  עִברִית  
  한국어  
  日本語  








Careers


















Close Menu







Careers






Chat






                                Home
                            



                                Career Areas
                            



                                Total Rewards
                            



                                Life@Nike
                            



                                Purpose
                            










Jordan Careers







Converse Careers










Language











Menu



Return to Previous Menu



Select a Langua

In [25]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
    """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):
        """
)

chain_extract = prompt_extract | llm
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [28]:
# convert str to JSON
from langchain_core.output_parsers import JsonOutputParser
json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content) # ignore 
json_res

[{'role': '12 Months Head Coach (Senior Store Manager), Nike - Brisbane Airport',
  'experience': None,
  'skills': None,
  'description': 'R-73333, Brisbane, Queensland, Australia, Retail Stores'},
 {'role': 'ATHLETE III',
  'experience': None,
  'skills': None,
  'description': 'R-74705, Nei Mongol, Nei Mongol, China, Retail Stores'},
 {'role': 'Account Engagement Specialist, Germany, Austria, & Switzerland (m/f/d)',
  'experience': None,
  'skills': None,
  'description': 'R-75330, Berlin, Berlin, Germany, Sales & Customer Service'},
 {'role': 'Account Engagement Specialist, Germany, Austria, & Switzerland (m/f/d)',
  'experience': None,
  'skills': None,
  'description': 'R-75331, Mainhausen, Hesse, Germany, Sales & Customer Service'},
 {'role': 'Account Engagement Specialist, UK & Ireland',
  'experience': None,
  'skills': None,
  'description': 'R-75163, London, London, United Kingdom, Sales & Customer Service'},
 {'role': 'Account Executive, SEA&I',
  'experience': None,
  'ski

In [29]:
type(json_res)

list